# 1. Find highest GeoTIFF resolution

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import _functions as pmf

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
raw_path = pmf.read_from_config(config_file,'raw_path')

## 1.1 Find all GeoTIFF file paths
We only need 1 file per data set, because we know that resolutions are consistent within a given data set.

In [4]:
import os

In [5]:
# Find the geotiffs and pre-filter everything not WorldClim
contents = []
for root, dirs, files in os.walk(raw_path):
    if files:
        for file in files:
            if file.lower().endswith('.tif'):
                contents.append(os.path.join(root,file))
                break

In [6]:
# Remove the extra WorldClim entries
worldclim_indices = [i for i, path in enumerate(contents) if 'worldclim' in path]
file_paths = [content for i,content in enumerate(contents) if i not in worldclim_indices[1:]]

## 1.2 Open geotiffs and find resolutions

In [7]:
import rasterio

In [8]:
def get_geotiff_resolution(file_path):
    with rasterio.open(file_path) as src:
        # Get the transform object from the dataset
        transform = src.transform

        # Extract x and y resolutions from the transform
        x_resolution = transform[0]
        y_resolution = transform[4]

    return x_resolution, y_resolution

In [9]:
x_resolutions = []
y_resolutions = []
for file in file_paths:
    x,y = get_geotiff_resolution(file)
    x_resolutions.append(x)
    y_resolutions.append(y)

In [22]:
x_res = min(x_resolutions) # smallest value == highest resolution
y_res = max(y_resolutions) # largest value, because all y-resolutions are negative by default

## 1.3 Add these to the config file
Assumes we have a file that does not yet contain the `x_res` and `y_res` values.

In [25]:
with open(config_file, 'a') as f:
    f.write(f'\nx_res | {x_res} | float # highest resolution in source datasets in x direction')
    f.write(f'\ny_res | {y_res} | float # highest resolution in source datasets in y direction')